In [9]:
import torch 
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F

%matplotlib inline

In [2]:
#load dataset 
#load  dataset to get list of strings  
words=open('names.txt','r').read().splitlines()

In [3]:
#some info of the dataset
print('total number of words in a dataset',len(words))
print('shortest word in the dataset',min(len(w)for w in words))
print('longest word in the dataset',max(len(w)for w in words))

total number of words in a dataset 32033
shortest word in the dataset 2
longest word in the dataset 15


In [4]:
#create dictionary
chars=sorted(list(set(''.join(words))))
#string to integer
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
#integer to string 
itos={i:s for s,i in stoi.items()}


E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see.

train: 2.2833642959594727

dev loss 2.277104616165161

test loss 2.277104616165161

so the model performs almost the same on all 3 dataset's 


In [5]:
def split(words):
    xs,ys=[],[]
    for w in words:
        chs=['.']+list(w)+['.']
        for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
            ix1=stoi[ch1]
            ix2=stoi[ch2]
            ix3=stoi[ch3]
            xs.append([ix1,ix2])
            ys.append(ix3)

    xs=torch.tensor(xs)
    ys=torch.tensor(ys)
    return xs,ys

In [6]:
random.shuffle(words)

#split 80% train ,10% dev,10% test 
train_size=int(0.8*len(words))
dev_size=int(0.9*len(words))

#train,dev,test datasets 
x_train,x_test=split(words[:train_size])
d_train,d_test=split(words[:dev_size])
t_train,t_test=split(words[dev_size:])


E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?  

so after 100 epochs we did get a little improvement 

bigram loss after 100 epochs: 2.4884581565856934

trigram loss after 100 epochs: 2.2833642959594727


i reduce the learning_rate to be 25,even that its a huge learning rate,probably adding a l1 reqularization would help,and for sure having a activation func(RELU). 


In [21]:
g = torch.Generator().manual_seed(2147483647)
#since we now using 2 char to predict the 3rd one we need to have [27*2,27]  
W_trigram=torch.rand((27*2,27),generator=g,requires_grad=True)

for k in range(100):
    #turn two char into one hot encoded vectors shape [N,2,27] cast to float 
    xenc=F.one_hot(x_train,num_classes=27).float()
    #view returns a new tensor with the same data diff shape [N,27*2] 
    logits=xenc.view(-1,27*2)@W_trigram
    #softmax 
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdims=True)
    #negative log likelihood + l2 regularization
    loss= -probs[torch.arange(x_test.shape[0]), x_test].log().mean() + 0.01 * (W_trigram**2).mean()
    print(loss.item())

    #backward pass 
    W_trigram.grad=None
    loss.backward()

    #gradient descent
    W_trigram.data+=-25*W_trigram.grad



3.4312031269073486
3.0970945358276367
2.885296106338501
2.7657711505889893
2.68744158744812
2.630394220352173
2.586986780166626
2.5529510974884033
2.5256099700927734
2.503190517425537
2.4844772815704346
2.468609571456909
2.454967498779297
2.4430928230285645
2.432645320892334
2.423366069793701
2.4150564670562744
2.4075615406036377
2.400758981704712
2.3945512771606445
2.3888590335845947
2.3836166858673096
2.378770351409912
2.3742752075195312
2.3700928688049316
2.3661904335021973
2.3625400066375732
2.359116315841675
2.3558998107910156
2.352870464324951
2.3500120639801025
2.3473100662231445
2.344752311706543
2.3423261642456055
2.340022325515747
2.3378305435180664
2.3357431888580322
2.3337526321411133
2.3318519592285156
2.3300347328186035
2.328295946121216
2.326629877090454
2.3250324726104736
2.3234987258911133
2.3220250606536865
2.320608139038086
2.319244384765625
2.3179306983947754
2.316664695739746
2.3154430389404297
2.3142640590667725
2.313124895095825
2.312023878097534
2.31095933914184

In [23]:
#evaluate func for  dev,test datasets  
def evaluate(features,labels):
    xenc=F.one_hot(features,num_classes=27).float()
    #we can think that this is like the first layer 
    logits = xenc.view(-1, 27*2) @ W_trigram 
    #softmax
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdims=True)
    #negative log likelihood + l2 regularization
    loss= -probs[torch.arange(features.shape[0]), labels].log().mean() + 0.001 * (W_trigram**2).mean()
    return loss.item()

In [24]:
dev_loss=evaluate(d_train,d_test)
test_loss=evaluate(d_train,d_test)
print('dev loss',dev_loss)
print('test loss',test_loss)


dev loss 2.277104616165161
test loss 2.277104616165161
